> Proszę zaproponować algorytm scalający k posortowanych list.

Ponieważ w zadaniu jest mowa o scalaniu list odsyłaczowych, listy źródłowe są modyfikowane.

##### Funkcja testująca poprawność algorytmu

In [1]:
import random


def test_merging(linked_list_creation,     # A class or a function which creates a linked list
                 linked_list_to_list,      # A function which converts a linked list to a list
                 merge_lists_fn,           # A function which performs merging on sorted linekd lists
                 samples=20,               # A number of repetitions (tests performed)
                 lists_counts=(0, 10),     # A number of lists created per a test
                 lists_lengths=(0, 10),    # A number of elements in each list
                 lists_values=(-100, 100), # A renge of which values will be picked to create a test linked list
                 failed_only=False         # A flag which indicates if all the results will be printed or only failed
                 ):
    passed = 0
    for i in range(1, samples + 1):
        # Create test data
        lists = []
        all_values = []
        input_lists = []
        for _ in range(random.randint(*lists_counts)):
            values = sorted(random.randint(*lists_values) for _ in range(random.randint(*lists_lengths)))
            input_lists.append(values)
            all_values.extend(values)
            ll_obj = linked_list_creation(values)
            lists.append(ll_obj)
        # Perform testing
        expected = sorted(all_values)
        result_ll = merge_lists_fn(lists)
        result = linked_list_to_list(result_ll)
        is_correct = result == expected
        passed += is_correct
        # Print information about a test performed
        if not failed_only or (failed_only and not is_correct):
            print(f'Test #{i}:')
            print('Input lists:')
            for idx, lst in enumerate(input_lists):
                print(f'\t{idx + 1}.  {lst}')
            print('Expected:', expected)
            print('Result:  ', result)
            print(f'Test {"PASSED" if is_correct else "FAILED"}')
            print(f'Passed to tested ratio: {passed}/{i}', end='\n\n')
    # Print final results
    print('===== Final results: =====')
    print(f'Total tests passed: {passed}/{samples}')
    print(f'An algorithm is {"CORRECT" if passed == samples else "WRONG"}')

##### Funkcja porównująca wydajność algorytmów

###### Uwaga

Poniższa funkcja nie sprawdza poprawności działania algorytmu. Przed przystąpieniem do porównania wydajności, sprawdź poprawność algorytmu, przy pomocy powyższej funkcji.

In [2]:
import random
import time


def compare_performance(data, *,
                        samples=20,               # A number of repetitions (tests performed)
                        lists_counts=(0, 10),     # A number of lists created per a test
                        lists_lengths=(0, 10),    # A number of elements in each list
                        lists_values=(-100, 100), # A renge of which values will be picked to create a test linked list
                        progress_interval=10      # A number of tests that will be performed between showing
                       ):
    times = dict.fromkeys(data.keys(), 0)
    max_algorithm_name_len = max(len(key) for key in data.keys())
    
    for i in range(1, samples + 1):
        # Create test data
        input_lists = []
        for _ in range(random.randint(*lists_counts)):
            values = sorted(random.randint(*lists_values) for _ in range(random.randint(*lists_lengths)))
            input_lists.append(values)
            
        # Perform time testing
        for name, (linked_list_creation, merge_lists_fn, globals_) in data.items():
            # Overwrite current global values to make an algorithm able to work (use a proper funciton)
            for obj_name, obj in globals_.items():
                globals()[obj_name] = obj
            # Create input linked lists
            lists = [linked_list_creation(values) for values in input_lists]
            # Perform test
            start_time = time.time()
            merge_lists_fn(lists)
            end_time = time.time()
            # Store test results
            times[name] += end_time - start_time
            
        # Print progress if set an interval
        if progress_interval > 0 and not i % progress_interval:
            print(f"===== Results after {i} tests: =====")
            for name in sorted(times, key=lambda name: times[name]):
                total_time = times[name]
                print(f"{name.rjust(max_algorithm_name_len)}   Total (in seconds): {total_time:.7f} \tAverage: {total_time/i:.7f}")
            print()
            
    # Print final results
    print("===== Final results: =====")
    print(f"Tests performed: {samples}")
    print("Total times (in seconds):")
    for name in sorted(times, key=lambda name: times[name]):
        total_time = times[name]
        print(f"\t{name.rjust(max_algorithm_name_len)}:   {total_time:.7f}")
    print("Average times (in seconds):")
    for name in sorted(times, key=lambda name: times[name]):
        total_time = times[name]
        print(f"\t{name.rjust(max_algorithm_name_len)}:   {total_time/samples:.7f}")

##### Słownik przechowujący algorytmy do porównania

In [3]:
algo_data = {}  # algo_data[<algorithm name>] = (<linked_list_creation>, <merge_lists_fn>, <global objects dict>)

## Sposób #1

Ten sposób polega na iterowaniu przez tablicę (listę), zawierającą wskaźniki do początków (wartowników) $ k $ posortowanych list, a następnie, łączeniu par list odsyłaczowych w jedną większą listę i dorzucanie jej z powrotem do połączenia później. Tzn. łączymy najpierw krótsze listy, tworząc z nich dłuższe i postępujemy tak, aż pozostanie nam jedna lista, będąca listą wynikową.

Jest to jeden z bardziej wydajnych sposobów, ktory przypomina nieco Merge Sorta dla list odsyłaczowych, polegającego na łączeniu serii naturalnych. Tutaj mamy od razu dane posortowane listy, więc sytuacja jest jeszcze łatwiejsza i polega jedynie na złączaniu list.

Zauważmy również, że poniższa implementacja nie potrzebuje rzadnego dodatkowego miejsca, a wszystkie operacje są wykonywane "w miejscu" (również przekazana sekwencja wskaźników do list odsyłaczowych zostaje zmodyfikowana)

### Implementacja algorytmu #1 (bez tworzenia dodatkowych struktur)

#### Implementacja obiektowa

In [4]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None
        

class LinkedList:
    def __init__(self, values: 'iterable' = None):
        self.head = self.tail = None
        self.length = 0
        if values: self.extend(values)
        
    def __iter__(self):
        curr = self.head
        while curr:
            yield curr.val
            curr = curr.next
    
    def __len__(self):
        return self.length
    
    def extend(self, values: 'iterable'):
        if values:
            iterator = iter(values)
            if not self:
                self.head = self.tail = Node(next(iterator))
            for val in iterator:
                self.tail.next = Node(val)
                self.tail = self.tail.next
            self.length += len(values)

            
def merge_sorted_lists_pair(ll1, ll2) -> LinkedList:
    # Create a result Linked List with a sentinel node
    result_ll = LinkedList([None])
    # Update a result linked list's length
    result_ll.length = len(ll1) + len(ll2)
    # Prepare pointers
    ll1_curr = ll1.head
    ll2_curr = ll2.head
    result_ll_curr = result_ll.head
    # Create a result (merged) linked list of two linked lists passed
    while ll1_curr and ll2_curr:
        if ll1_curr.val < ll2_curr.val:
            result_ll_curr.next = ll1_curr
            ll1_curr = ll1_curr.next
        else:
            result_ll_curr.next = ll2_curr
            ll2_curr = ll2_curr.next
        result_ll_curr = result_ll_curr.next
    # Link the remaining parts
    if ll1_curr:
        result_ll_curr.next = ll1_curr
    else:
        result_ll_curr.next = ll2_curr
    # Unlink a sentinel node from a result linked list
    result_ll.head = result_ll.head.next
    # Return a linked list created
    return result_ll
    
    
def merge_sorted_lists(lls) -> LinkedList:
    # Return an empty linked list if there are no input lists or the first list if there is
    # only one linked list
    if not lls: return LinkedList()
    # Else, if there are at least 2 lists, merge them to the one list
    interval = 1
    amount = len(lls)
    while interval < amount:
        for i in range(0, amount - interval, 2 * interval):
            lls[i] = merge_sorted_lists_pair(lls[i], lls[i + interval])
        interval *= 2
    
    # Return a result of merging all the lists
    return lls[0]

##### Zapisujemy dane do późniejszego benchmarku (nie jest to częścią algorytmu)

In [5]:
data = (LinkedList, merge_sorted_lists, {
    'merge_sorted_lists_pair': merge_sorted_lists_pair,
    'merge_sorted_lists': merge_sorted_lists,
    'LinkedList': LinkedList,
    'Node': Node
})
algo_data['(Sposób #1) (Implementacja #1) W miejscu obiektowo'] = data

###### Kilka testów

In [6]:
test_merging(LinkedList, list, merge_sorted_lists)
# test_merging(LinkedList, list, merge_sorted_lists,
#             samples=1000, lists_counts=(0, 100), lists_lengths=(0, 100),
#             failed_only=True)

Test #1:
Input lists:
	1.  [54]
	2.  []
	3.  [-91, -5, 99]
	4.  [-51, -38, -27, -6, -2, 9, 29, 37, 92]
	5.  []
	6.  [-78, -61, -56, 2, 24, 34, 78, 99]
Expected: [-91, -78, -61, -56, -51, -38, -27, -6, -5, -2, 2, 9, 24, 29, 34, 37, 54, 78, 92, 99, 99]
Result:   [-91, -78, -61, -56, -51, -38, -27, -6, -5, -2, 2, 9, 24, 29, 34, 37, 54, 78, 92, 99, 99]
Test PASSED
Passed to tested ratio: 1/1

Test #2:
Input lists:
	1.  [-89, -67, 41, 55, 68, 69]
	2.  [-52, -37, -20, -11, 2, 34, 35, 55, 74]
	3.  [-79, -64, -13, -8, 12, 35, 56]
	4.  [-61, -52]
	5.  [-91, -87, -42, -26, -12, 21, 45, 46, 67, 86]
	6.  [-96]
	7.  [-82, -37, -11, -9, 59, 63, 88, 88, 93]
	8.  [-76, -48, -6, 2, 5, 29, 36, 59, 88]
	9.  [-59]
	10.  [-52, 1]
Expected: [-96, -91, -89, -87, -82, -79, -76, -67, -64, -61, -59, -52, -52, -52, -48, -42, -37, -37, -26, -20, -13, -12, -11, -11, -9, -8, -6, 1, 2, 2, 5, 12, 21, 29, 34, 35, 35, 36, 41, 45, 46, 55, 55, 56, 59, 59, 63, 67, 68, 69, 74, 86, 88, 88, 88, 93]
Result:   [-96, -91, -89, 

#### Implementacja funkcyjna

In [7]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None


def create_linked_list(values: 'iterable' = None) -> 'linked list head (sentinel)':
    head = Node()  # A sentinel node
    if not values: return head
    head.next = curr = Node(values[0])
    for i in range(1, len(values)):
        curr.next = Node(values[i])
        curr = curr.next
    return head


def merge_sorted_lists_pair(ll1_head, ll2_head) -> 'linked list head (sentinel)':
    ll_res_head = Node()  # Store a sentinel node

    ll_res_curr = ll_res_head
    ll1_curr = ll1_head.next
    ll2_curr = ll2_head.next

    while ll1_curr and ll2_curr:
        if ll1_curr.val < ll2_curr.val:
            ll_res_curr.next = ll1_curr
            ll1_curr = ll1_curr.next
        else:
            ll_res_curr.next = ll2_curr
            ll2_curr = ll2_curr.next
        ll_res_curr = ll_res_curr.next

    # Link a remaining part to a result linked list
    if ll1_curr:
        ll_res_curr.next = ll1_curr
    else:
        ll_res_curr.next = ll2_curr

    return ll_res_head


def merge_sorted_lists(lls) -> 'linked list head (sentinel)':
    # Return an empty linked list sentinel node if there are no linked lists in lls iterable
    if not lls: return Node()
    # Else, if there are at least 2 lists, merge them to the one list
    interval = 1
    amount = len(lls)
    while interval < amount:
        for i in range(0, amount - interval, 2 * interval):
            lls[i] = merge_sorted_lists_pair(lls[i], lls[i + interval])
        interval *= 2
    
    # Return a result of merging all the lists
    return lls[0]

##### Funkcja pomocnicza, konwertująca listę odsyłaczową do listy Pythonowej

In [8]:
def linked_list_to_list(ll_head: 'linked list head (sentinel)') -> list:
    result = []
    curr = ll_head.next
    while curr:
        result.append(curr.val)
        curr = curr.next
    return result

##### Zapisujemy dane do późniejszego benchmarku (nie jest to częścią algorytmu)

In [9]:
data = (create_linked_list, merge_sorted_lists, {
    'merge_sorted_lists_pair': merge_sorted_lists_pair,
    'merge_sorted_lists': merge_sorted_lists,
    'create_linked_list': create_linked_list,
    'linked_list_to_list': linked_list_to_list,
    'Node': Node
})
algo_data['(Sposób #1) (Implementacja #1) W miejscu funkcyjnie'] = data

###### Kilka testów

In [10]:
test_merging(create_linked_list, linked_list_to_list, merge_sorted_lists)
# test_merging(LinkedList, list, merge_sorted_lists,
#             samples=1000, lists_counts=(0, 100), lists_lengths=(0, 100),
#             failed_only=True)

Test #1:
Input lists:
	1.  [-22]
	2.  [-33, -12, -10, 97]
	3.  [-91, -85, -62, -12, -7, -6, 54, 87, 95]
	4.  [-72, -4, 0, 25]
	5.  [-55, -22, -18, 18, 22, 64, 66, 94]
	6.  [47, 54, 90]
	7.  [-85, 48, 68, 92]
	8.  [-66, -53, -9, 26, 38, 45, 65, 66, 85, 92]
	9.  [-76]
	10.  [-79, -75, -25, -13, 15, 37, 37]
Expected: [-91, -85, -85, -79, -76, -75, -72, -66, -62, -55, -53, -33, -25, -22, -22, -18, -13, -12, -12, -10, -9, -7, -6, -4, 0, 15, 18, 22, 25, 26, 37, 37, 38, 45, 47, 48, 54, 54, 64, 65, 66, 66, 68, 85, 87, 90, 92, 92, 94, 95, 97]
Result:   [-91, -85, -85, -79, -76, -75, -72, -66, -62, -55, -53, -33, -25, -22, -22, -18, -13, -12, -12, -10, -9, -7, -6, -4, 0, 15, 18, 22, 25, 26, 37, 37, 38, 45, 47, 48, 54, 54, 64, 65, 66, 66, 68, 85, 87, 90, 92, 92, 94, 95, 97]
Test PASSED
Passed to tested ratio: 1/1

Test #2:
Input lists:
	1.  [-86, 42, 54, 91]
	2.  [-93, -78, -71, -62, -58, -58, -48, -46, -14, 90]
	3.  [-92, -9, 61]
	4.  [-95, -67, -44, 25, 37, 53]
	5.  [-89, -84, 39, 58, 67, 76, 9

### Implementacja algorytmu #2 (z użyciem kolejki, lepsza, bo nie tworzy za każdym razem nowej tablicy)

#### Implementacja obiektowa

In [11]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None
        
        
class Queue:
    def __init__(self, values: 'iterable' = None):
        self.head = self.tail = None
        self.length = 0
        if values: self.enqueue_many(values)
    
    def __len__(self):
        return self.length
    
    def enqueue(self, val: object):
        node = Node(val)
        if not self:
            self.head = self.tail = node
        else:
            self.tail.next = node
            self.tail = node
        self.length += 1
        
    def dequeue(self) -> object:
        if not self:
            raise IndexError(f'dequeue from an empty {self.__class__.__name__}')
        removed = self.head.val
        if len(self) == 1:
            self.head = self.tail = None
        else:
            self.head = self.head.next
        self.length -= 1
        return removed
    
    def enqueue_many(self, values: 'iterable'):
        if values:
            iterator = iter(values)
            if not self:
                self.head = self.tail = Node(next(iterator))
            for val in iterator:
                self.tail.next = Node(val)
                self.tail = self.tail.next
            self.length += len(values)
            
            
class LinkedList:
    def __init__(self, values: 'iterable' = None):
        self.head = self.tail = None
        self.length = 0
        if values: self.extend(values)
        
    def __iter__(self):
        curr = self.head
        while curr:
            yield curr.val
            curr = curr.next
    
    def __len__(self):
        return self.length
    
    def extend(self, values: 'iterable'):
        if values:
            iterator = iter(values)
            if not self:
                self.head = self.tail = Node(next(iterator))
            for val in iterator:
                self.tail.next = Node(val)
                self.tail = self.tail.next
            self.length += len(values)
            
            
def merge_sorted_lists_pair(ll1, ll2) -> LinkedList:
    # Create a result Linked List with a sentinel node
    result_ll = LinkedList([None])
    # Update a result linked list's length
    result_ll.length = len(ll1) + len(ll2)
    # Prepare pointers
    ll1_curr = ll1.head
    ll2_curr = ll2.head
    result_ll_curr = result_ll.head
    # Create a result (merged) linked list of two linked lists passed
    while ll1_curr and ll2_curr:
        if ll1_curr.val < ll2_curr.val:
            result_ll_curr.next = ll1_curr
            ll1_curr = ll1_curr.next
        else:
            result_ll_curr.next = ll2_curr
            ll2_curr = ll2_curr.next
        result_ll_curr = result_ll_curr.next
    # Link the remaining parts
    if ll1_curr:
        result_ll_curr.next = ll1_curr
    else:
        result_ll_curr.next = ll2_curr
    # Unlink a sentinel node from a result linked list
    result_ll.head = result_ll.head.next
    # Return a linked list created
    return result_ll
    
    
def merge_sorted_lists(lls) -> LinkedList:
    # Return an empty linked list if there are no input lists or the first list if there is
    # only one linked list
    if not lls: return LinkedList()
    # Else, if there are at least 2 lists, merge them to the one list
    queue = Queue(lls)
    while len(queue) > 1:
        # Merge adjacent linked lists and add them to the queue
        ll1 = queue.dequeue()
        ll2 = queue.dequeue()
        queue.enqueue(merge_sorted_lists_pair(ll1, ll2))
    
    # Return a result of merging all the lists
    return queue.dequeue()

##### Zapisujemy dane do późniejszego benchmarku (nie jest to częścią algorytmu)

In [12]:
data = (LinkedList, merge_sorted_lists, {
    'merge_sorted_lists_pair': merge_sorted_lists_pair,
    'merge_sorted_lists': merge_sorted_lists,
    'LinkedList': LinkedList,
    'Queue': Queue,
    'Node': Node
})
algo_data['(Sposób #1) (Implementacja #2) Z kolejką obiektowo'] = data

###### Kilka testów

In [13]:
test_merging(LinkedList, list, merge_sorted_lists)
# test_merging(LinkedList, list, merge_sorted_lists,
#             samples=1000, lists_counts=(0, 100), lists_lengths=(0, 100),
#             failed_only=True)

Test #1:
Input lists:
	1.  [-69, -55, -13, 2, 11, 31, 97, 98]
Expected: [-69, -55, -13, 2, 11, 31, 97, 98]
Result:   [-69, -55, -13, 2, 11, 31, 97, 98]
Test PASSED
Passed to tested ratio: 1/1

Test #2:
Input lists:
	1.  [-87, -54, 16, 83]
	2.  [-33, -9]
	3.  [-99, -85, -69, -60, -32, -10, 32]
	4.  [-32, 9, 72, 79]
	5.  []
	6.  [-49]
Expected: [-99, -87, -85, -69, -60, -54, -49, -33, -32, -32, -10, -9, 9, 16, 32, 72, 79, 83]
Result:   [-99, -87, -85, -69, -60, -54, -49, -33, -32, -32, -10, -9, 9, 16, 32, 72, 79, 83]
Test PASSED
Passed to tested ratio: 2/2

Test #3:
Input lists:
	1.  [-66, -51, -24, -15, -6, -3, 38, 43]
	2.  [-88, -62, -2, 9, 35, 63, 88]
Expected: [-88, -66, -62, -51, -24, -15, -6, -3, -2, 9, 35, 38, 43, 63, 88]
Result:   [-88, -66, -62, -51, -24, -15, -6, -3, -2, 9, 35, 38, 43, 63, 88]
Test PASSED
Passed to tested ratio: 3/3

Test #4:
Input lists:
	1.  [-99, -88, -36, -35, -1, 17, 47, 47, 52, 58]
	2.  []
	3.  [-88, -11, 70]
	4.  [16, 45]
	5.  [-72, -47, -41, -40, 6, 30,

#### Implementacja funkcyjna (Nie ma sensu dodawać. Kolejka funkcyjna to tragedia.)

## Sposób #2

W tym sposobie wykorzystujemy kolejkę priorytetową. W kolejce umieszczamy kolejno listy odsyłaczowe z priorytetem równym wartości ich obecnie pierwszego węzła. Następnie wyciągamy z kolejki (będącej w rzeczywistości minimalnym kopcem binarnym) i dopinamy pierwszy węzeł tej listy na koniec listy wynikowej, a w liście ściągniętej z kolejki, przesuwamy początkowy wskaźnik na następny element. Jeżeli tym elementem, na który przesunęliśmy wskażnik, jest None, oznacza to, że dana lista odsyłaczowa się skończyła i już jej nie rozważamy (nie dodajemy do kolejki priorytetowej). Jeżeli natomiast lista nie jest jeszcze wyczerpana, wstawiamy do kolejki priorytetowej tę listę (po przesunięciu się o 1 węzeł) wraz z priorytetem, będącym wartością obecnie pierwszego węzła. 

Sposób ten jest nieco bardziej skomplikowany od poprzednich i algorytm działa wolniej oraz potrzebuje więcej pamięci, ze względu na umieszczenie wskaźników do wszystkich list w kolejce priorytetowej.

#### Implementacja obiektowa

In [14]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None
        
        
class LinkedList:
    def __init__(self, values: 'iterable' = None):
        self.head = self.tail = None
        self.length = 0
        if values: self.extend(values)
        
    def __iter__(self):
        curr = self.head
        while curr:
            yield curr.val
            curr = curr.next
    
    def __len__(self):
        return self.length
    
    def extend(self, values: 'iterable'):
        if values:
            iterator = iter(values)
            if not self:
                self.head = self.tail = Node(next(iterator))
            for val in iterator:
                self.tail.next = Node(val)
                self.tail = self.tail.next
            self.length += len(values)
        

class MinPriorityQueue:
    def __init__(self):
        self._heap = []
        
    def __len__(self):
        return len(self._heap)
        
    def insert(self, priority: int, val: object):
        if not isinstance(priority, int):
            raise TypeError(f"priority must be 'int', not {str(type(priority))[7:-1]}")
        # Add a value as the last node of our Complete Binary Tree
        self._heap.append((priority, val))
        # Fix heap in order to satisfy a min-heap property
        self._heapify_up(len(self) - 1)
        
    # Removes the first value in a priority queue (of the lowest priority)
    def poll(self):
        if not self:
            raise IndexError(f'poll from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self._heap[0][1]
        # Place the last leaf in the root position
        last = self._heap.pop()
        if len(self) > 0:
            self._heap[0] = last
            # Fix a heap in order to stisfy a max-heap property
            self._heapify_down(0, len(self))
        return removed
            
    @staticmethod
    def _parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def _left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def _right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def _swap(self, i, j):
        self._heap[i], self._heap[j] = self._heap[j], self._heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):  # O(log n)
        while curr_idx > end_idx:
            parent_idx = self._parent_idx(curr_idx)
            if self._heap[curr_idx][0] < self._heap[parent_idx][0]:
                self._swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        while True:
            l = self._left_child_idx(curr_idx)
            r = self._right_child_idx(curr_idx)
            largest_idx = curr_idx

            if l < end_idx:
                if self._heap[l][0] < self._heap[curr_idx][0]: 
                    largest_idx = l
                if r < end_idx and self._heap[r][0] < self._heap[largest_idx][0]:
                    largest_idx = r

            if largest_idx != curr_idx:
                self._swap(curr_idx, largest_idx)
                curr_idx = largest_idx
            else:
                break
    
    
def merge_sorted_lists(lls) -> LinkedList:
    # Return an empty linked list if there are no input lists or the first list if there is
    # only one linked list
    if not lls: return LinkedList()
    
    # Else, if there are at least 2 lists, merge them to the one list
    # Prepare variables for merging
    result_ll = LinkedList([None])  # Create a sentinel node
    result_ll_curr = result_ll.tail
    pq = MinPriorityQueue()
    for ll in lls:
        # If is not empty, insert a linked list to the min priority queue
        if ll:
            pq.insert(ll.head.val, ll.head)
            result_ll.length += len(ll)
            
    # Loop till a priority queue has more than 1 element (linked list) stored
    while len(pq) > 1:
        curr_ll = pq.poll()
        result_ll_curr.next = curr_ll
        curr_ll = curr_ll.next
        result_ll_curr = result_ll_curr.next
        # If there is still some remaining part of the current linked list, add it to the queue
        if curr_ll:
            pq.insert(curr_ll.val, curr_ll)
            
    # If there is still something remaining in the priority queue, link this list to the result
    if pq:
        result_ll_curr.next = pq.poll()
    
    # If a loop is finished, remove a sentinel node and return a result linked list
    result_ll.head = result_ll.head.next
    return result_ll

##### Zapisujemy dane do późniejszego benchmarku (nie jest to częścią algorytmu)

In [15]:
data = (LinkedList, merge_sorted_lists, {
    'merge_sorted_lists': merge_sorted_lists,
    'MinPriorityQueue': MinPriorityQueue,
    'LinkedList': LinkedList,
    'Node': Node
})
algo_data['(Sposób #2) Z kolejką priorytetową obiektowo'] = data

###### Kilka testów

In [16]:
test_merging(LinkedList, list, merge_sorted_lists)
# test_merging(LinkedList, list, merge_sorted_lists,
#             samples=100, lists_counts=(0, 100), lists_lengths=(0, 100),
#             failed_only=True)

Test #1:
Input lists:
	1.  [-87, -66, -35, -11, -7, 27, 58, 70, 81, 99]
	2.  [-86, -84, -69, -33, -30, -7, -2, 90]
Expected: [-87, -86, -84, -69, -66, -35, -33, -30, -11, -7, -7, -2, 27, 58, 70, 81, 90, 99]
Result:   [-87, -86, -84, -69, -66, -35, -33, -30, -11, -7, -7, -2, 27, 58, 70, 81, 90, 99]
Test PASSED
Passed to tested ratio: 1/1

Test #2:
Input lists:
	1.  [-84, -23, -8, -2, 16, 27, 27, 40, 70, 84]
	2.  [-86, -48, -45, -35, -24, 5, 20, 25, 63, 82]
	3.  [-82, 71]
	4.  [-88, -67, -61, -49, -36, 11, 21, 38, 67]
	5.  [-80, -72, -70, -67, 5, 21, 57, 80]
Expected: [-88, -86, -84, -82, -80, -72, -70, -67, -67, -61, -49, -48, -45, -36, -35, -24, -23, -8, -2, 5, 5, 11, 16, 20, 21, 21, 25, 27, 27, 38, 40, 57, 63, 67, 70, 71, 80, 82, 84]
Result:   [-88, -86, -84, -82, -80, -72, -70, -67, -67, -61, -49, -48, -45, -36, -35, -24, -23, -8, -2, 5, 5, 11, 16, 20, 21, 21, 25, 27, 27, 38, 40, 57, 63, 67, 70, 71, 80, 82, 84]
Test PASSED
Passed to tested ratio: 2/2

Test #3:
Input lists:
	1.  [-98,

#### Implementacja funkcyjna 

In [17]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None


def create_linked_list(values: 'iterable' = None) -> 'linked list head (sentinel)':
    head = Node()  # A sentinel node
    if not values: return head
    head.next = curr = Node(values[0])
    for i in range(1, len(values)):
        curr.next = Node(values[i])
        curr = curr.next
    return head


_left = lambda i: 2 * i + 1
_right = lambda i: 2 * i + 2
_parent = lambda i: (i - 1) // 2
        
        
def insert_to_min_priority_queue(queue: list, priority: int, val: object):
    if not isinstance(priority, int):
        raise TypeError(f"priority must be 'int', not {str(type(priority))[7:-1]}")
    # Add a value as the last node of our Complete Binary Tree
    queue.append((priority, val))
    # Fix heap in order to satisfy a max-heap property
    _heapify_up(queue, len(queue) - 1)
    
    
def poll_from_min_priority_queue(queue: list) -> object:
    if not queue:
        raise IndexError(f'poll from an empty Priority Queue')
    # Store a value to be returned
    removed = queue[0][1]
    # Place the last leaf in the root position
    last = queue.pop()
    if queue:
        queue[0] = last
        # Fix a heap in order to stisfy a max-heap property
        _heapify_down(queue, 0, len(queue))
    return removed

    
def _swap(queue: list, i, j):
    queue[i], queue[j] = queue[j], queue[i]
    
    
def _heapify_up(queue: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index' = 0):
    while curr_idx > end_idx:
        parent_idx = _parent(curr_idx)
        if queue[curr_idx][0] < queue[parent_idx][0]:
            _swap(queue, curr_idx, parent_idx)
        curr_idx = parent_idx
            
    
def _heapify_down(queue: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index'):
    while True:
        i = _left(curr_idx)
        j = _right(curr_idx)
        k = curr_idx
        
        if i < end_idx:
            if queue[i][0] < queue[k][0]:
                k = i
            if j < end_idx and queue[j][0] < queue[k][0]:
                k = j
                
        if k == curr_idx: return
        # Swap the current with the largest child
        queue[curr_idx], queue[k] = queue[k], queue[curr_idx]
        curr_idx = k
        
        
def merge_sorted_lists(lls) -> 'linked list head (sentinel)':
    # Return an empty linked list (sentinel node) if there is no linked list in a lls iterable
    if not lls: return Node()
    
    # Else, if there are at least 2 lists, merge them to the one list
    # Prepare variables for merging
    result_ll = Node()
    result_curr = result_ll
    pq = []
    for ll_head in lls:
        # If is not empty, insert a linked list to the min priority queue
        if ll_head.next:
            insert_to_min_priority_queue(pq, ll_head.next.val, ll_head.next)
            
    # Loop till a priority queue has more than 1 element (linked list) stored
    while len(pq) > 1:
        curr_head = poll_from_min_priority_queue(pq)
        result_curr.next = curr_head
        curr_head = curr_head.next
        result_curr = result_curr.next
        # If there is still some remaining part of the current linked list, add it to the queue
        if curr_head:
            insert_to_min_priority_queue(pq, curr_head.val, curr_head)
            
    # If there is still something remaining in the priority queue, link this list to the result
    if pq:
        result_curr.next = poll_from_min_priority_queue(pq)

    return result_ll

##### Funkcja pomocnicza, konwertująca listę odsyłaczową do listy Pythonowej

In [18]:
def linked_list_to_list(ll_head: 'linked list head (sentinel)') -> list:
    result = []
    curr = ll_head.next
    while curr:
        result.append(curr.val)
        curr = curr.next
    return result

##### Zapisujemy dane do późniejszego benchmarku (nie jest to częścią algorytmu)

In [19]:
data = (create_linked_list, merge_sorted_lists, {
    'create_linked_list': create_linked_list,
    'Node': Node,
    '_left': _left,
    '_right': _right,
    '_parent': _parent,
    'insert_to_min_priority_queue': insert_to_min_priority_queue,
    'poll_from_min_priority_queue': poll_from_min_priority_queue,
    '_swap': _swap,
    '_heapify_up': _heapify_up,
    '_heapify_down': _heapify_down,
    'merge_sorted_lists': merge_sorted_lists,
    'linked_list_to_list': linked_list_to_list,
})
algo_data['(Sposób #2) Z kolejką priorytetową funkcyjnie'] = data

###### Kilka testów

In [20]:
test_merging(create_linked_list, linked_list_to_list, merge_sorted_lists)
# test_merging(create_linked_list, linked_list_to_list, merge_sorted_lists,
#             samples=100, lists_counts=(0, 100), lists_lengths=(0, 100),
#             failed_only=True)

Test #1:
Input lists:
Expected: []
Result:   []
Test PASSED
Passed to tested ratio: 1/1

Test #2:
Input lists:
	1.  [9, 97]
	2.  []
Expected: [9, 97]
Result:   [9, 97]
Test PASSED
Passed to tested ratio: 2/2

Test #3:
Input lists:
	1.  [-97, -51, -51, -41, -31, -11, 19, 73, 94, 98]
	2.  []
	3.  [-85, -66, -13, -9, 38, 39, 46, 70]
	4.  [-35, 14, 84]
	5.  [-85, -26, 94]
	6.  [-93, -80, -53, -51, 77, 98]
	7.  [-95, -77, -68, -64, -52, -7, 57, 83, 92]
Expected: [-97, -95, -93, -85, -85, -80, -77, -68, -66, -64, -53, -52, -51, -51, -51, -41, -35, -31, -26, -13, -11, -9, -7, 14, 19, 38, 39, 46, 57, 70, 73, 77, 83, 84, 92, 94, 94, 98, 98]
Result:   [-97, -95, -93, -85, -85, -80, -77, -68, -66, -64, -53, -52, -51, -51, -51, -41, -35, -31, -26, -13, -11, -9, -7, 14, 19, 38, 39, 46, 57, 70, 73, 77, 83, 84, 92, 94, 94, 98, 98]
Test PASSED
Passed to tested ratio: 3/3

Test #4:
Input lists:
Expected: []
Result:   []
Test PASSED
Passed to tested ratio: 4/4

Test #5:
Input lists:
	1.  [-49, 8, 9, 31]

## Sposób #3

Sposób podobny do #1, ale tym razem łączymy zawsze 2 najkrótsze pod względem liczby węzłów listy.

#### Implementacja funkcyjna

In [21]:
class MinPriorityQueue:
    def __init__(self):
        self._heap = []
        
    def __len__(self):
        return len(self._heap)
        
    def insert(self, priority: int, val: object):
        if not isinstance(priority, int):
            raise TypeError(f"priority must be 'int', not {str(type(priority))[7:-1]}")
        # Add a value as the last node of our Complete Binary Tree
        self._heap.append((priority, val))
        # Fix heap in order to satisfy a min-heap property
        self._heapify_up(len(self) - 1)
        
    # Removes the first value in a priority queue (of the lowest priority)
    def poll(self):
        if not self:
            raise IndexError(f'poll from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self._heap[0]
        # Place the last leaf in the root position
        last = self._heap.pop()
        if len(self) > 0:
            self._heap[0] = last
            # Fix a heap in order to stisfy a max-heap property
            self._heapify_down(0, len(self))
        return removed
            
    @staticmethod
    def _parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def _left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def _right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def _swap(self, i, j):
        self._heap[i], self._heap[j] = self._heap[j], self._heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):  # O(log n)
        while curr_idx > end_idx:
            parent_idx = self._parent_idx(curr_idx)
            if self._heap[curr_idx][0] < self._heap[parent_idx][0]:
                self._swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        while True:
            l = self._left_child_idx(curr_idx)
            r = self._right_child_idx(curr_idx)
            largest_idx = curr_idx

            if l < end_idx:
                if self._heap[l][0] < self._heap[curr_idx][0]: 
                    largest_idx = l
                if r < end_idx and self._heap[r][0] < self._heap[largest_idx][0]:
                    largest_idx = r

            if largest_idx != curr_idx:
                self._swap(curr_idx, largest_idx)
                curr_idx = largest_idx
            else:
                break
                
                
class Node:
    def __init__(self, val=None):
        self.next = None
        self.val = val


def get_length(head):
    length = 0
    curr = head.next
    while curr:
        length += 1
        curr = curr.next
    return length


def _merge(head1, head2):
    curr1 = head1.next
    curr2 = head2.next
    head = tail = Node(None)

    while curr1 and curr2:
        if curr1.val < curr2.val:
            tail.next = curr1
            curr1 = curr1.next
        else:
            tail.next = curr2
            curr2 = curr2.next
        tail = tail.next

    if curr1: tail.next = curr1
    else:     tail.next = curr2

    return head

    
def merge_lists(L):
    n = len(L)
    if not L: return Node()
    if n == 1: return L[0]
    # Get length of each list and add this list with its length
    # as a priority to the minimum priority queue
    pq = MinPriorityQueue()
    i = 0
    while i < n:
        pq.insert(get_length(L[i]), L[i])
        i += 1
    # In a loop, merge lists of the lowest length together and add
    # a resulting list back to the priority queue
    while len(pq) > 1:
        length1, head1 = pq.poll()
        length2, head2 = pq.poll()
        new_head = _merge(head1, head2)
        new_length = length1 + length2
        if pq:
            pq.insert(new_length, new_head)
            i += 1
        else:
            return new_head

##### Funkcja pomocnicza, konwertująca listę odsyłaczową do listy Pythonowej

In [22]:
def linked_list_to_list(ll_head: 'linked list head (sentinel)') -> list:
    result = []
    curr = ll_head.next
    while curr:
        result.append(curr.val)
        curr = curr.next
    return result

def create_linked_list(values: 'iterable' = None) -> 'linked list head (sentinel)':
    head = Node()  # A sentinel node
    if not values: return head
    head.next = curr = Node(values[0])
    for i in range(1, len(values)):
        curr.next = Node(values[i])
        curr = curr.next
    return head

##### Zapisujemy dane do późniejszego benchmarku (nie jest to częścią algorytmu)

In [23]:
data = (create_linked_list, merge_lists, {
    'Node': Node,
    'get_length': get_length,
    '_merge': _merge,
    'merge_lists': merge_lists,
    'linked_list_to_list': linked_list_to_list,
    'create_linked_list': create_linked_list,
    'MinPriorityQueue': MinPriorityQueue
})
algo_data['(Sposób #3) Z kolejką priorytetową po długości'] = data

###### Kilka testów

In [24]:
test_merging(create_linked_list, linked_list_to_list, merge_lists)
# test_merging(create_linked_list, linked_list_to_list, merge_sorted_lists,
#             samples=100, lists_counts=(0, 100), lists_lengths=(0, 100),
#             failed_only=True)

Test #1:
Input lists:
	1.  [-94, -88, -83, -79, -69, -60, -55, 22, 38, 46]
	2.  [-83, -66, -64, -47, -37, 13, 20, 24, 32, 72]
	3.  [-97, -67, -45, 73, 78, 95]
	4.  [-57, -36, -6, 5, 34, 53, 81]
	5.  [-98, -98, -69, -67, -29, -25, -10, 11, 61]
	6.  []
	7.  [-73, -10, 32]
Expected: [-98, -98, -97, -94, -88, -83, -83, -79, -73, -69, -69, -67, -67, -66, -64, -60, -57, -55, -47, -45, -37, -36, -29, -25, -10, -10, -6, 5, 11, 13, 20, 22, 24, 32, 32, 34, 38, 46, 53, 61, 72, 73, 78, 81, 95]
Result:   [-98, -98, -97, -94, -88, -83, -83, -79, -73, -69, -69, -67, -67, -66, -64, -60, -57, -55, -47, -45, -37, -36, -29, -25, -10, -10, -6, 5, 11, 13, 20, 22, 24, 32, 32, 34, 38, 46, 53, 61, 72, 73, 78, 81, 95]
Test PASSED
Passed to tested ratio: 1/1

Test #2:
Input lists:
	1.  [29]
	2.  [-91, -74, -47, -36, 24, 56, 60, 73]
	3.  [-96, -54, -53, -49, -40, -27, 25, 61, 70, 74]
	4.  [-84, -71, -67, 45, 58, 66, 76]
	5.  [-85, -63, -32, -10, -8, 58, 92]
	6.  [-84, -70, -56, 9, 94]
	7.  [-47, -12, 56]
	8.  [-

	10.  [16, 50]
Expected: [-99, -95, -95, -88, -87, -83, -80, -79, -77, -75, -70, -67, -66, -63, -61, -59, -56, -52, -51, -49, -37, -36, -34, -31, -29, -27, -26, -25, -23, -18, -15, 0, 2, 4, 16, 17, 18, 20, 35, 47, 50, 55, 59, 59, 62, 63, 63, 64, 69, 70, 77, 82, 85, 91, 92, 93, 93, 96, 96, 97, 100]
Result:   [-99, -95, -95, -88, -87, -83, -80, -79, -77, -75, -70, -67, -66, -63, -61, -59, -56, -52, -51, -49, -37, -36, -34, -31, -29, -27, -26, -25, -23, -18, -15, 0, 2, 4, 16, 17, 18, 20, 35, 47, 50, 55, 59, 59, 62, 63, 63, 64, 69, 70, 77, 82, 85, 91, 92, 93, 93, 96, 96, 97, 100]
Test PASSED
Passed to tested ratio: 17/17

Test #18:
Input lists:
	1.  [-4, 40, 54, 78]
	2.  [5, 51, 74]
	3.  [-87, -72, -42, -25, -20, 12, 62]
	4.  [-99, -69, -8, 15, 34, 37, 81, 93, 95]
	5.  [-37, -29, -24, -13, -5, 69, 82]
Expected: [-99, -87, -72, -69, -42, -37, -29, -25, -24, -20, -13, -8, -5, -4, 5, 12, 15, 34, 37, 40, 51, 54, 62, 69, 74, 78, 81, 82, 93, 95]
Result:   [-99, -87, -72, -69, -42, -37, -29, -25,

# Porównanie wydajności

In [25]:
compare_performance(algo_data,
                    samples=100,
                    lists_counts=(0, 1_000),
                    lists_lengths=(0, 100),
                    lists_values=(-1_000, 1_000),
                    progress_interval=5
                   )

===== Results after 5 tests: =====
 (Sposób #1) (Implementacja #1) W miejscu obiektowo   Total (in seconds): 0.3041115 	Average: 0.0608223
(Sposób #1) (Implementacja #1) W miejscu funkcyjnie   Total (in seconds): 0.3190944 	Average: 0.0638189
 (Sposób #1) (Implementacja #2) Z kolejką obiektowo   Total (in seconds): 0.3463099 	Average: 0.0692620
     (Sposób #3) Z kolejką priorytetową po długości   Total (in seconds): 0.4014821 	Average: 0.0802964
      (Sposób #2) Z kolejką priorytetową funkcyjnie   Total (in seconds): 1.0236990 	Average: 0.2047398
       (Sposób #2) Z kolejką priorytetową obiektowo   Total (in seconds): 1.5471632 	Average: 0.3094326

===== Results after 10 tests: =====
(Sposób #1) (Implementacja #1) W miejscu funkcyjnie   Total (in seconds): 0.6556153 	Average: 0.0655615
 (Sposób #1) (Implementacja #1) W miejscu obiektowo   Total (in seconds): 0.6826472 	Average: 0.0682647
 (Sposób #1) (Implementacja #2) Z kolejką obiektowo   Total (in seconds): 0.6826975 	Average: 0.

===== Results after 70 tests: =====
(Sposób #1) (Implementacja #1) W miejscu funkcyjnie   Total (in seconds): 3.7223146 	Average: 0.0531759
 (Sposób #1) (Implementacja #1) W miejscu obiektowo   Total (in seconds): 3.7365427 	Average: 0.0533792
 (Sposób #1) (Implementacja #2) Z kolejką obiektowo   Total (in seconds): 3.8165803 	Average: 0.0545226
     (Sposób #3) Z kolejką priorytetową po długości   Total (in seconds): 4.5947332 	Average: 0.0656390
      (Sposób #2) Z kolejką priorytetową funkcyjnie   Total (in seconds): 11.9914386 	Average: 0.1713063
       (Sposób #2) Z kolejką priorytetową obiektowo   Total (in seconds): 18.1701338 	Average: 0.2595733

===== Results after 75 tests: =====
(Sposób #1) (Implementacja #1) W miejscu funkcyjnie   Total (in seconds): 3.9884555 	Average: 0.0531794
 (Sposób #1) (Implementacja #1) W miejscu obiektowo   Total (in seconds): 4.0123091 	Average: 0.0534975
 (Sposób #1) (Implementacja #2) Z kolejką obiektowo   Total (in seconds): 4.1699085 	Average: